In [1]:
import pandas as pd
import mysql.connector

In [2]:
connection = mysql.connector.connect(
    host= 'localhost',         
    user= 'root',             
    password= '123456', 
    database = 'day_52'     
)
cursor = connection.cursor()

In [3]:
cursor.execute("""
CREATE TABLE customers (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL, 
    city VARCHAR(200) NOT NULL
);
""")

In [4]:
cursor.execute("""
INSERT INTO customers (id, name, city) VALUES
(1, 'Alice', 'Hanoi'),
(2, 'Bob', 'Ho Chi Minh'),
(3, 'Charlie', 'Hanoi');
""")

connection.commit()

In [5]:
cursor.execute("""
CREATE TABLE orders (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    total INTEGER NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
);
""")

In [6]:
cursor.execute("""
INSERT INTO orders (id, customer_id, order_date, total) VALUES
(1, 1, '2024-01-10', 500),
(2, 1, '2024-03-01', 800),
(3, 2, '2024-02-20', 1200),
(4, 3, '2024-01-15', 200);
""")

connection.commit()

In [7]:
cursor.execute("""
CREATE TABLE products (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    price INTEGER NOT NULL
);
""")

In [8]:
cursor.execute("""
INSERT INTO products (id, name, price) VALUES
(1, 'Laptop', 1500),
(2, 'Mouse', 50),
(3, 'Keyboard', 100),
(4, 'Monitor', 300);
""")

connection.commit()

In [9]:
cursor.execute("""
CREATE TABLE order_items (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);
""")

In [10]:
cursor.execute("""
INSERT INTO order_items (id, order_id, product_id, quantity) VALUES
(1, 1, 2, 2),
(2, 1, 3, 1),
(3, 2, 1, 1),
(4, 2, 4, 1),
(5, 3, 1, 1);
""")

connection.commit()

In [11]:
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    department VARCHAR(200) NOT NULL
);
""")

In [12]:
cursor.execute("""
INSERT INTO employees (id, name, department) VALUES
(1, 'David', 'Sales'),
(2, 'Emma', 'Support'),
(3, 'Frank', 'Sales');
""")

connection.commit()

In [13]:
cursor.execute("""
CREATE TABLE order_assignments (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    employee_id INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (employee_id) REFERENCES employees(id)
);
""")

In [14]:
cursor.execute("""
INSERT INTO order_assignments (id, order_id, employee_id) VALUES
(1, 1, 1),
(2, 2, 3),
(3, 3, 2);
""")

connection.commit()

### Bài 1

In [3]:
cursor.execute("""
CREATE INDEX city_index
ON customers (city)
;
""")

### Bài 2

In [4]:
cursor.execute("""
CREATE INDEX customer_index
ON orders (customer_id)
;
""")

### Bài 3

In [5]:
cursor.execute("""
CREATE UNIQUE INDEX name_index
ON products (name)
;
""")

### Bài 4

In [6]:
cursor.execute("""
CREATE INDEX order_product_index
ON order_items (product_id, order_id)
;
""")

### Bài 5

In [ ]:
cursor.execute("""
CREATE INDEX order_product_index
ON order_items (product_id, order_id)
;
""")

### Bài 6

In [7]:
cursor.execute("EXPLAIN SELECT * FROM products WHERE price > 1000")
print("Hiệu suất truy vấn KHÔNG có index:")
for row in cursor.fetchall():
    print(row)

Hiệu suất truy vấn KHÔNG có index:
(1, 'SIMPLE', 'products', None, 'ALL', None, None, None, None, 4, 33.33, 'Using where')


In [8]:
cursor.execute("CREATE INDEX idx_product_price ON products(price)")
connection.commit()

cursor.execute("EXPLAIN SELECT * FROM products WHERE price > 1000")
print("\nHiệu suất truy vấn CÓ index:")
for row in cursor.fetchall():
    print(row)


Hiệu suất truy vấn CÓ index:
(1, 'SIMPLE', 'products', None, 'range', 'idx_product_price', 'idx_product_price', '4', None, 1, 100.0, 'Using index condition')


### Bài 7

In [9]:
cursor.execute("""
DROP INDEX city_index
ON customers;
""")

### Bài 8

In [10]:
cursor.execute("""
CREATE INDEX department_index
ON employees (department)
;
""")

In [11]:
query = ("""
SELECT * 
 FROM employees
""")

data_8 = pd.read_sql_query(query, connection)
data_8

C:\Users\Apollonir\AppData\Local\Temp\ipykernel_2320\2348000317.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_8 = pd.read_sql_query(query, connection)


,id,name,department
0,1,David,Sales
1,2,Emma,Support
2,3,Frank,Sales


### Bài 9

In [12]:
import time

cursor.execute("DROP INDEX idx_product_price ON products")
connection.commit()

start_time = time.time()

for _ in range(1000):
    cursor.execute("SELECT * FROM products WHERE price > 1000")
    cursor.fetchall()

end_time = time.time()
no_index_time = end_time - start_time
print(f"Thời gian truy vấn không có index: {no_index_time:.6f} giây")

Thời gian truy vấn không có index: 0.285116 giây


In [13]:
cursor.execute("CREATE INDEX idx_product_price ON products(price)")
connection.commit()

start_time = time.time()

for _ in range(1000):
    cursor.execute("SELECT * FROM products WHERE price > 1000")
    cursor.fetchall()

end_time = time.time()
with_index_time = end_time - start_time
print(f"Thời gian truy vấn có index: {with_index_time:.6f} giây")

Thời gian truy vấn có index: 0.280940 giây


In [14]:
improvement = (no_index_time - with_index_time) / no_index_time * 100
print(f"Cải thiện hiệu suất: {improvement:.2f}%")

Cải thiện hiệu suất: 1.46%


### Bài 10

In [15]:
cursor.execute("""
CREATE INDEX date_index
ON orders (order_date)
;
""")